In [4]:
import os
import cv2
import numpy as np
import random

import keras
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Conv2D, concatenate, Embedding, Reshape, Flatten, Activation
from keras.optimizers import SGD

#constants to resize image 
HEIGHT = 256
WIDTH = 512

HEIGHT_REQUIRED_PORTION = 0.4
WIDTH_REQUIRED_PORTION = 0.5

YAW_ADJ_DEGREES = 35

height_from = int(HEIGHT * (1 -HEIGHT_REQUIRED_PORTION))
width_from = int((WIDTH - WIDTH * WIDTH_REQUIRED_PORTION) / 2)
width_to = width_from + int(WIDTH_REQUIRED_PORTION * WIDTH)

#get a list of files
img_dir = os.path.join('archive', 'train_label')
images = [f.split('.')[0] for f in os.listdir(img_dir) if f.endswith(".png")]

random.shuffle(images)

X = [] #images
Y = [] #expected steering for this image
for example in images:
    img_path = img_dir+'/'+example+'.png'
    image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (WIDTH,HEIGHT))
    image = image[height_from:,width_from:width_to]
    canny = cv2.Canny(image,50,150)
    X.append(canny[:, :, None] / 255)
    y = example.split('_')[-1]
    y = float(y)/YAW_ADJ_DEGREES 
    Y.append(y)

X = np.array(X)
Y = np.array(Y)

#rescale Y
true_max = max(abs(min(Y)),max(Y))
Y *= (1.0/true_max)

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())  
model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(1))

model.summary()

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['MSE'])


model.fit(X, Y, batch_size=32, shuffle=False, epochs=10, validation_split=0.2)

predictions = model.predict(X)
print("Prediction min: ",predictions.min()," Prediction max: ",predictions.max())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 144, 320, 64)      640       
                                                                 
 activation_5 (Activation)   (None, 144, 320, 64)      0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 72, 160, 64)      0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 160, 64)       36928     
                                                                 
 activation_6 (Activation)   (None, 72, 160, 64)       0         
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 80, 64)       0         
 2D)                                                  

In [6]:
model.save(".\model\lane_model", overwrite=True,include_optimizer=True,
    save_format=None, signatures=None, options=None, save_traces=True)

INFO:tensorflow:Assets written to: lane_model_360x640_02_20\assets


INFO:tensorflow:Assets written to: lane_model_360x640_02_20\assets


In [9]:
for i in range(100):
    print('Actual y: ', Y[i], " Predicted y: ",predictions[i])
    cv2.imshow('image',X[i])
    cv2.waitKey(0)

Actual y:  0.0  Predicted y:  [-0.03976538]
Actual y:  0.5142857142857142  Predicted y:  [0.4549619]
Actual y:  -0.19999999999999998  Predicted y:  [-0.28337932]
Actual y:  -0.4285714285714285  Predicted y:  [-0.492609]
Actual y:  -0.5428571428571428  Predicted y:  [-0.56820077]
Actual y:  0.0  Predicted y:  [-0.00050758]
Actual y:  -0.657142857142857  Predicted y:  [-0.70105684]
Actual y:  -0.8285714285714285  Predicted y:  [-0.84249157]
Actual y:  0.14285714285714282  Predicted y:  [0.18693627]
Actual y:  0.0857142857142857  Predicted y:  [0.05673605]
Actual y:  0.0  Predicted y:  [-0.033412]
Actual y:  0.3428571428571428  Predicted y:  [0.3264245]
Actual y:  0.19999999999999998  Predicted y:  [0.17185964]
Actual y:  -0.657142857142857  Predicted y:  [-0.69270307]
Actual y:  0.4285714285714285  Predicted y:  [0.44365215]
Actual y:  0.0  Predicted y:  [-0.08101199]
Actual y:  0.6  Predicted y:  [0.6132954]
Actual y:  -0.8857142857142856  Predicted y:  [-0.8849879]
Actual y:  -0.457142